In [1]:
import simtk.openmm.app as app
import simtk.openmm as omm
import simtk.unit as u

import parmed as pmd

In [2]:
top_file = '../MD_exps/VHP_exp/pdb/vhp1ww.top' 
pdb_file = './vhp1ww.pdb'

In [3]:
output_traj="output.dcd" 
output_log="output.log" 
output_cm=None 
report_time=10*u.picoseconds 
sim_time=10*u.nanoseconds

In [4]:
if top_file:
    pdb = pmd.load_file(top_file, xyz = pdb_file)
    system = pdb.createSystem(nonbondedMethod=app.CutoffNonPeriodic,
            nonbondedCutoff=1.0*u.nanometer, constraints=app.HBonds,
            implicitSolvent=app.OBC1)
else:
    pdb = pmd.load_file(pdb_file)
    forcefield = app.ForceField('amber99sbildn.xml', 'amber99_obc.xml')
    system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.CutoffNonPeriodic,
            nonbondedCutoff=1.0*u.nanometer, constraints=app.HBonds)

dt = 0.002*u.picoseconds
integrator = omm.LangevinIntegrator(300*u.kelvin, 91.0/u.picosecond, dt)
integrator.setConstraintTolerance(0.00001)

In [5]:
pdb.positions[0]

Quantity(value=Vec3(x=11.522, y=-46.675, z=-30.412), unit=angstrom)

In [6]:
import random

In [7]:
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(random.choice(pdb.get_coordinates()))

In [11]:
simulation.context.setPositions(pdb.positions)
app.PDBFile.writeFile(pdb.topology, simulation.context.getState(getPositions=True) .getPositions(), open('vhp_test.pdb', 'w'))
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

simulation.context.setPositions(random.choice(pdb.get_coordinates())) 
app.PDBFile.writeFile(pdb.topology, simulation.context.getState(getPositions=True) .getPositions(), open('vhp_test.pdb', 'w'))
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

In [10]:
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

121763144.0 kJ/mol


In [9]:
state = simulation.context.getState(getPositions=True) 

In [10]:
app.PDBFile.writeFile(pdb.topology, state.getPositions(), open('vhp_mini2.pdb', 'w'))

In [ ]:
report_freq = int(report_time/dt)
simulation.context.setVelocitiesToTemperature(10*u.kelvin, random.randint(1, 10000))
simulation.reporters.append(app.DCDReporter(output_traj, report_freq))
if output_cm:
    simulation.reporters.append(ContactMapReporter(output_cm, report_freq))
simulation.reporters.append(app.StateDataReporter(output_log,
        report_freq, step=True, time=True, speed=True,
        potentialEnergy=True, temperature=True, totalEnergy=True))
simulation.reporters.append(app.CheckpointReporter('checkpnt.chk', report_freq))

# if check_point:
#     simulation.loadCheckpoint(check_point)
nsteps = int(sim_time/dt)
simulation.step(nsteps)

In [14]:
x = 10*u.picoseconds/ (0.002*u.picoseconds)
x

5000.0